В данном ноутбуке будет реализовано три способа action recognition
1. Классификация видео используя Video Resnet
2. Классификация видео используя Rethinking Spatiotemporal Feature Learning
3. Покадровая класссификация изображений


In [1]:
!pip install vidaug >> None
!pip install av >> None
!pip install transformers >> None

# Imports

In [1]:
import os
import random
import time
import warnings

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn.functional as F
from torchvision.io import read_video
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchsummary import summary
import av

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import f1_score
from vidaug import augmentors as va
from transformers import AutoProcessor, AutoModel
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

warnings.simplefilter("ignore", UserWarning)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load data

In [2]:
video_path = '/content/drive/MyDrive/Colab Notebooks/Курс компьютерное зрение/videos/'
data_path = '/content/drive/MyDrive/Colab Notebooks/Курс компьютерное зрение/data/data.csv'

In [3]:
df = pd.read_csv(data_path)
df[:3]

,name_video,label
0,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tap dancing
1,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tap dancing
2,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tap dancing


In [4]:
# кодируем лейблы
encoder = OrdinalEncoder()
df['target'] = encoder.fit_transform(np.array(df['label']).reshape(-1, 1))
df

,name_video,label,target
0,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tap dancing,14.0
1,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tap dancing,14.0
2,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tap dancing,14.0
3,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tap dancing,14.0
4,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tap dancing,14.0
...,...,...,...
205,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tango dancing,13.0
206,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tango dancing,13.0
207,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tango dancing,13.0
208,/content/drive/MyDrive/Colab Notebooks/Курс ко...,tango dancing,13.0


In [5]:
# константы и настройки
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
RAND = 10
num_features = df.target.nunique()
input_frame_size = (224, 224)
count_frames = 16
model_lr = 1e-5
epochs = 15
test_size = 0.2
batch_size = 16

In [8]:
class OurDataset(Dataset):
    def __init__(self, data, is_train = False):
        self.df = data
        self.path_video = ".."
        self.is_train = is_train
        # установим частоту аугментации видео
        freq_aug = lambda aug: va.Sometimes(0.7, aug)
        # параметры аугментации
        self.aug = va.Sequential([
            va.RandomRotate(degrees=(-25, 25)),
            freq_aug(va.HorizontalFlip()),
            freq_aug(va.Salt()),
            freq_aug(va.Pepper()),
            freq_aug(va.GaussianBlur(0.2))])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        target = row['target']

        path = row['name_video']
        # считываем видео, делаем преобразования
        video, _, _ = read_video(path, pts_unit="sec")

        if len(video) > 0:
            video = video[:count_frames]
            video = video.numpy()

            if self.is_train:
                video = self.aug(video)

            video = torch.Tensor(video)
            resize_transform = transforms.Resize(input_frame_size)
            video_resized = torch.stack([resize_transform(
                frame.permute(2, 0, 1)).permute(1, 2, 0) for frame in video])
            video_normalized = video_resized.permute(3, 0, 1, 2)
            tensor_3d = video_normalized / 255
        else:
            tensor_3d = torch.empty(3, 16, input_frame_size)

        label = torch.tensor(target).long()
        return tensor_3d, label

In [7]:
dataset_train = OurDataset(df)
dataset_train[0][0].shape

torch.Size([3, 16, 224, 224])

In [8]:
# создаем трнировочную и валидационную выборки
train, val = train_test_split(df, test_size=test_size, random_state=RAND, stratify=df['target'])
train_dataset = OurDataset(train.reset_index(), is_train=True)
val_dataset = OurDataset(val.reset_index(), is_train=False)
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
valid_loader = DataLoader(val_dataset,
                          batch_size=batch_size)

#

# VIDEO RESNET


In [67]:
model = models.video.mc3_18(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, num_features)
model.to(device)
device

Downloading: "https://download.pytorch.org/models/mc3_18-a90a0ba3.pth" to /root/.cache/torch/hub/checkpoints/mc3_18-a90a0ba3.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 81.6MB/s]


device(type='cuda')

In [68]:
model_optimizer = optim.AdamW(model.parameters(), model_lr)
criterion = nn.CrossEntropyLoss()

In [71]:
for epoch in range(epochs):

    model.train()

    train_loss = []
    for i, (batch, targets) in enumerate(tqdm(train_loader, desc=f"Epoch: {epoch}")):
        model_optimizer.zero_grad()
        batch = batch.to(device)
        targets = targets.to(device)

        logits = model.forward(batch)

        loss = criterion(logits, targets)
        loss.backward()
        model_optimizer.step()

        train_loss.append(loss.item())

    print('Training loss:', np.mean(train_loss))

    model.eval()

    val_loss = []
    val_targets = []
    val_preds = []
    for i, (batch, targets) in enumerate(tqdm(valid_loader, desc=f"Epoch: {epoch}")):
        with torch.no_grad():

            batch = batch.to(device)
            targets = targets.to(device)

            logits = model.forward(batch)

            loss = criterion(logits, targets)

            val_loss.append(loss.item())
            val_targets.extend(targets.cpu().numpy())
            val_preds.extend(logits.argmax(axis=1).cpu().numpy())

    print('Val loss:', np.mean(val_loss))
    print('F1:', f1_score(val_targets, val_preds, average='macro'))

Epoch: 0: 100%|██████████| 7/7 [01:18<00:00, 11.26s/it]


Training loss: 2.784828083855765


Epoch: 0: 100%|██████████| 2/2 [00:40<00:00, 20.01s/it]


Val loss: 2.860278606414795
F1: 0.020634920634920634


Epoch: 1: 100%|██████████| 7/7 [01:18<00:00, 11.14s/it]


...

Val loss: 8.144031286239624
F1: 0.018611746758199845


Epoch: 14: 100%|██████████| 7/7 [01:18<00:00, 11.16s/it]


Training loss: 2.7338344029017856


Epoch: 14: 100%|██████████| 2/2 [00:05<00:00,  2.59s/it]

Val loss: 8.207681655883789
F1: 0.019001610305958132


# Rethinking Spatiotemporal Feature Learning

In [19]:
model = models.video.s3d(pretrained=True)
# заменяем последний слой для классификации
model.classifier =  nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Conv3d(1024, num_features, kernel_size=1, stride=1, bias=True),
        )
model.to(device)
device

device(type='cuda')

In [10]:
model_optimizer = optim.AdamW(model.parameters(), model_lr)
scheduler = lr_scheduler.StepLR(model_optimizer, step_size=3, gamma=0.8)
criterion = nn.CrossEntropyLoss()

In [28]:
for epoch in range(epochs):

    model.train()

    train_loss = []
    for i, (batch, targets) in enumerate(tqdm(train_loader, desc=f"Epoch: {epoch}")):
        model_optimizer.zero_grad()
        batch = batch.to(device)
        targets = targets.to(device)


        logits = model.forward(batch)

        loss = criterion(logits, targets)
        loss.backward()
        model_optimizer.step()

        train_loss.append(loss.item())

    print('Training loss:', np.mean(train_loss))

    model.eval()

    val_loss = []
    val_targets = []
    val_preds = []
    for i, (batch, targets) in enumerate(tqdm(valid_loader, desc=f"Epoch: {epoch}")):
        with torch.no_grad():

            batch = batch.to(device)
            targets = targets.to(device)

            logits = model.forward(batch)

            loss = criterion(logits, targets)

            val_loss.append(loss.item())
            val_targets.extend(targets.cpu().numpy())
            val_preds.extend(logits.argmax(axis=1).cpu().numpy())

    print('Val loss:', np.mean(val_loss))
    print('F1:', f1_score(val_targets, val_preds, average='macro'))

Epoch: 0: 100%|██████████| 11/11 [02:17<00:00, 12.50s/it]


Training loss: 2.791024598208341


Epoch: 0: 100%|██████████| 3/3 [00:05<00:00,  1.97s/it]


Val loss: 2.7162607510884604
F1: 0.062380952380952384


Epoch: 1: 100%|██████████| 11/11 [02:16<00:00, 12.37s/it]


...

Val loss: 2.7260993321736655
F1: 0.11148148148148147


Epoch: 14: 100%|██████████| 11/11 [02:22<00:00, 12.96s/it]


Training loss: 2.782319437373768


Epoch: 14: 100%|██████████| 3/3 [00:06<00:00,  2.32s/it]

Val loss: 2.7264417012532554
F1: 0.11333333333333333


**Видно, что вторая модель показывает результаты получше чем Video Resnet. Общее же низкое качество метрики и обучения в целом вероятно связано с небольшой выборкой видео семплов (300 шт)**

# Покадровая классификация

Для реализации покадровой классификации будем использовать стандартный подход классификации каждого кадра как отдельного изображения. Для этого необходимо изменить подготовку данных

In [30]:
class FrameDataset(Dataset):
    def __init__(self, data, is_train = False):
        self.df = data
        self.path_video = ".."
        self.is_train = is_train

        # параметры аугментации
        if is_train:
          self.aug =  A.Compose([
              A.Resize(height=input_frame_size[0],
                       width=input_frame_size[0],
                       always_apply=True),
              A.Rotate([-45,45], p=1),
              A.RandomRain(p=0.05),
              A.HorizontalFlip(p=0.5),
              A.GaussianBlur(blur_limit=(3, 5), p=0.02),
              A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5],),
              ToTensorV2(),
          ])
        else:
            self.aug =  A.Compose([
                A.Resize(height=input_frame_size[0],
                         width=input_frame_size[0],
                         always_apply=True),
                A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5],),
                ToTensorV2(),
            ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        target = row['target']

        path = row['name_video']
        # считываем видео, берем случайный кадр и аугментируем
        video, _, _ = read_video(path, pts_unit="sec")


        frames_count = video.shape[0]
        random_frame = video[random.randint(0, frames_count - 1)].numpy()
        aug_frame = self.aug(image=random_frame)['image']


        label = torch.tensor(target).long()
        return aug_frame, label

In [31]:
dataset_train = FrameDataset(df)
dataset_train[0][0].shape

torch.Size([3, 224, 224])

In [32]:
# создаем трнировочную и валидационную выборки
train, val = train_test_split(df, test_size=test_size, random_state=RAND, stratify=df['target'])
train_dataset = FrameDataset(train.reset_index(), is_train=True)
val_dataset = FrameDataset(val.reset_index(), is_train=False)
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
valid_loader = DataLoader(val_dataset,
                          batch_size=batch_size)

In [18]:
# для классификации будем использовать модель efficientnet
model = models.efficientnet.efficientnet_v2_l(pretrained=True)
model.classifier = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                                nn.Linear(1280, num_features))
if torch.cuda.is_available():
    model.cuda()

In [ ]:
summary(model, (3, 224, 224))

In [20]:
model_optimizer = optim.AdamW(model.parameters(), model_lr)
criterion = nn.CrossEntropyLoss()

In [33]:
for epoch in range(epochs):

    model.train()

    train_loss = []
    for i, (batch, targets) in enumerate(tqdm(train_loader, desc=f"Epoch: {epoch}")):
        model_optimizer.zero_grad()
        batch = batch.to(device)
        targets = targets.to(device)


        logits = model.forward(batch)

        loss = criterion(logits, targets)
        loss.backward()
        model_optimizer.step()

        train_loss.append(loss.item())

    print('Training loss:', np.mean(train_loss))

    model.eval()

    val_loss = []
    val_targets = []
    val_preds = []
    for i, (batch, targets) in enumerate(tqdm(valid_loader, desc=f"Epoch: {epoch}")):
        with torch.no_grad():

            batch = batch.to(device)
            targets = targets.to(device)

            logits = model.forward(batch)

            loss = criterion(logits, targets)

            val_loss.append(loss.item())
            val_targets.extend(targets.cpu().numpy())
            val_preds.extend(logits.argmax(axis=1).cpu().numpy())

    print('Val loss:', np.mean(val_loss))
    print('F1:', f1_score(val_targets, val_preds, average='macro'))

Epoch: 0: 100%|██████████| 11/11 [00:22<00:00,  2.03s/it]


Training loss: 2.7828515442934902


Epoch: 0: 100%|██████████| 3/3 [00:13<00:00,  4.42s/it]


Val loss: 2.7211809158325195
F1: 0.03636363636363636


Epoch: 1: 100%|██████████| 11/11 [00:18<00:00,  1.65s/it]


...

Val loss: 2.646632194519043
F1: 0.05384615384615385


Epoch: 14: 100%|██████████| 11/11 [00:18<00:00,  1.66s/it]


Training loss: 2.4308996200561523


Epoch: 14: 100%|██████████| 3/3 [00:04<00:00,  1.35s/it]

Val loss: 2.6722238063812256
F1: 0.07265031265031266
